![GCAT | Genomes for Life](images/logo-GCAT.png)

# Filter locations

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pylab import *
%matplotlib inline

## Load tables

- MICA
- Participants

In [2]:
mica = pd.read_csv("/home/labs/dnalab/share/lims/R/gcat-cohort/output/export/QUESTIONARI/data.csv", low_memory=False)
participants = pd.read_csv("/home/labs/dnalab/share/lims/R/gcat-cohort/output/export/Participants/data.csv")

all = participants
all = pd.merge(all, mica, on=['entity_id'])

## Load country codes

In [17]:
countries = [
    'ANDORRA',
    'ANGOLA',
    'ARGENTINA',
    'ARMENIA',
    'AUSTRIA',
    'AUSTRALIA',
    'BELGIUM',
    'BOLIVIA',
    'BULGARIA',
    'BRAZIL',
    'CANADA',
    'CHILE',
    'COLOMBIA',
    'CUBA',
    'DENMARK',
    'DOMINICAN REPUBLIC',
    'ECUADOR',
    'EGYPT',
    'EL SALVADOR',
    'EQUADOR',
    'EQUATORIAL GUINEA',
    'FINLAND',
    'FRANCE',
    'GERMANY',
    'GUATEMALA',
    'GUINEA ECUATORIAL',
    'ICELAND',
    'IRAK',
    'IRAN',
    'IRELAND',
    'ITALY',
    'MACEDONIA',
    'MEXICO',
    'MOROCCO',
    'NETHERLANDS',
    'PERU',
    'POLONIA',
    'PORTUGAL',
    'PUERTO RICO',
    'ROMANIA',
    'RUSSIA',
    'SERBIA',
    'SPAIN',
    'SWEDEN',
    'SWITZERLAND',
    'TUNISIA',
    'UNITED KINGDOM',
    'URUGUAY',
    'VENEZUELA'
]

## Select variables of interest

In [18]:
columns = [
    'entity_id',
    'MUNICIPIO_NACIMIENTO',
    'MUNICIPIO_RESIDENCIA_OTRO',
    'MUNICIPIO_NACIMIENTO_OTRO',
    'MUNICIPIO_RESIDENCIA',
    'PAIS_NACIMIENTO',
    'PAIS_NACIMIENTO_OTRO',
]

all = all.ix[:,columns]

## Province of residence

In [19]:
filter = all.copy()
regional_codes = {'08': 'BARCELONA', '17': 'GIRONA', '25': 'LLEIDA', '43': 'TARRAGONA'}

filter['PROVINCIA_RESIDENCIA'] = filter['MUNICIPIO_RESIDENCIA'].str.extract('(\d\d)')
filter = filter.replace({'PROVINCIA_RESIDENCIA': regional_codes})

## Country of birth

In [20]:
country_codes = {1: 'SPAIN', 2: None}
filter = filter.replace({'PAIS_NACIMIENTO': country_codes})
filter.loc[
    filter['PAIS_NACIMIENTO'] != 'SPAIN', 'PAIS_NACIMIENTO'] = filter.loc[
    filter['PAIS_NACIMIENTO'] != 'SPAIN', 'PAIS_NACIMIENTO_OTRO']

## Town of birth

In [21]:
filter.loc[
    filter['MUNICIPIO_NACIMIENTO'].isnull(), 'MUNICIPIO_NACIMIENTO'] = filter.loc[
    filter['MUNICIPIO_NACIMIENTO'].isnull(), 'MUNICIPIO_NACIMIENTO_OTRO']

filter['MUNICIPIO_NACIMIENTO'] = filter['MUNICIPIO_NACIMIENTO'].str.upper()

## Curate foreign country names

In [22]:
a_la_llista = True & ~filter.PAIS_NACIMIENTO_OTRO.isin(countries)

filter['LUGAR_NACIMIENTO.OTRO.PAIS'] = filter['PAIS_NACIMIENTO_OTRO']
filter['LUGAR_NACIMIENTO'] = filter['PAIS_NACIMIENTO']
output = filter.loc[filter['PAIS_NACIMIENTO_OTRO'].notnull() & a_la_llista,
           ['entity_id', 'LUGAR_NACIMIENTO', 'LUGAR_NACIMIENTO.OTRO.PAIS']]

output.to_csv('../output/locations/other_countries.csv', index=False)

## Write curated data

In [23]:
columns = [
    'entity_id',
    'PAIS_NACIMIENTO',
    'MUNICIPIO_NACIMIENTO',
    'PROVINCIA_RESIDENCIA'
]
data = filter[columns]
data.to_csv('../output/locations/data.csv', index=False)